In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from stl import mesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Mediapipe Face Mesh 모듈 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 얼굴 랜드마크 좌표를 추출하는 함수
def extract_face_landmarks(image):
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.1, min_tracking_confidence=0.1) as face_mesh:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image_rgb)

        landmarks = []
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                for landmark in face_landmarks.landmark:
                    landmarks.append((landmark.x, landmark.y, landmark.z))
        return landmarks

# 얼굴 랜드마크 좌표를 STL 형식으로 저장하는 함수
def save_landmarks_to_stl(landmarks, stl_filename):
    vertices = np.array(landmarks)

    # Define the 3D mesh using numpy-stl
    mesh_data = mesh.Mesh(np.zeros(len(landmarks), dtype=mesh.Mesh.dtype))
    for i, vertex in enumerate(landmarks):
        mesh_data.vectors[i] = vertex

    # Save the STL file
    mesh_data.save(stl_filename)

# 얼굴 랜드마크를 3D Mesh에 입히고 시각화하는 함수
def visualize_3d_mesh_with_texture(landmarks):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # 얼굴 랜드마크를 연결하여 Mesh 생성
    mesh_data = mesh.Mesh(np.zeros(len(landmarks), dtype=mesh.Mesh.dtype))
    for i, vertex in enumerate(landmarks):
        mesh_data.vectors[i] = vertex

    # STL 파일로 Mesh 저장
    stl_filename = 'output_mesh.stl'
    mesh_data.save(stl_filename)

    # Mesh 시각화
    ax.plot_trisurf(mesh_data.x.flatten(), mesh_data.y.flatten(), mesh_data.z.flatten(), triangles=mesh_data.vectors)

    # 시점 변경
    ax.view_init(elev=-100, azim=-90)  # elev은 고도, azim은 방위각

    plt.show()

# 이미지 로드
image_path = 'test_img.jpg'
image = cv2.imread(image_path)

# 얼굴 랜드마크 추출
face_landmarks = extract_face_landmarks(image)

# 얼굴 랜드마크를 3D Mesh에 입히고 시각화
if face_landmarks:
    visualize_3d_mesh_with_texture(face_landmarks)
else:
    print('No face landmarks detected.')
